In [55]:
from Bio import SeqIO
from Bio.Seq import Seq
import os

In [56]:
os.getcwd()

'C:\\Users\\Joshua_Dietrich\\OneDrive - UW-Madison\\JD - Josh Dietrich\\Machine_Learning\\KDC_specificity'

## ZmPDC libraries

In [57]:
file_dir = 'C:\\Users\\Joshua_Dietrich\\OneDrive - UW-Madison\\JD - Josh Dietrich\\Dietrich_E-Lab_Notebook\\Sequencing_results\\2023\\2023_1212_G4W\\Pfleger_G4W_raw_reads'
filename = 'Pfleger_G4W_3_ZmPDC_lib_arced.fastq'
reads = [r for r in SeqIO.parse(os.path.join(file_dir,filename),'fastq')]

In [58]:
Zmpos1 = 'TTTTAATGAT' # upstream 10 bases of first mutation
Zmpos2 = 'GGGCGATTCT' # upstream 10 bases of second mutation
Zmpos3 = 'CGGTTATACG'
Zmpos4 = 'ACTTGTTAAG'
Zm_positions = [Seq(Zmpos1),Seq(Zmpos2),Seq(Zmpos3),Seq(Zmpos4)]
min_qscore = 20 # min quality score for counting a mutation

In [59]:
found_codons = [dict() for p in Zm_positions]
residues = [dict() for p in Zm_positions]
read_dict = dict()
mut_combo_dict = dict()
for r in reads:
    # print(r.id)
    read_dict[r.id] = [None] * len(Zm_positions) # initialize a dict that maps a read to its mutated codons
    for idx,(p,d) in enumerate(zip(Zm_positions,found_codons)):
        # Loop through each mutation position
        if r.seq.find(p) != -1: # If the read actually has the current codon in it
            curr_read = r.seq
            curr_read_q_scores = r.letter_annotations["phred_quality"]
        elif r.seq.reverse_complement().find(p) != -1: # If the read is the reverse direction
            curr_read = r.seq.reverse_complement()
            curr_read_q_scores = list(reversed(r.letter_annotations["phred_quality"]))
        else:
            continue
        curr_codon_idx = curr_read.find(p)+len(p)
        curr_codon = curr_read[curr_codon_idx:curr_codon_idx+3]
        curr_q_scores = curr_read_q_scores[curr_codon_idx:curr_codon_idx+3]
        
        # Check if the codon has a high enough q score, also checks if q scores correctly found
        if len(curr_q_scores) == 3 and min(curr_q_scores) >= min_qscore:
            # print(idx)
            # print(curr_codon,curr_q_scores)
            read_dict[r.id][idx] = str(curr_codon)
            if curr_codon in found_codons[idx]: # If the current codon has already been found
                found_codons[idx][str(curr_codon)] +=1
                residues[idx][str(Seq(curr_codon).translate())] +=1
            else:
                found_codons[idx][str(curr_codon)] = 1 # If the current codon has not yet been found
                residues[idx][str(Seq(curr_codon).translate())] = 1
    curr_mut_combo = ''.join([str(Seq(x).translate()) if x is not None else 'X' for x in read_dict[r.id]])
    if curr_mut_combo in mut_combo_dict:
        mut_combo_dict[curr_mut_combo] += 1
    else:
        mut_combo_dict[curr_mut_combo] = 1
mut_combo_dict.pop('XXXX',None) # Remove uninformative read data from count
    # print(idx,found_codons[idx],residues[idx])

204

In [60]:
for idx,d in enumerate(found_codons):
    print(len(d),'unique codons found for position',idx+1)

34 unique codons found for position 1
33 unique codons found for position 2
34 unique codons found for position 3
34 unique codons found for position 4


In [61]:
for idx,r in enumerate(residues):
    print(len(r),'unique residues found for position',idx+1)

21 unique residues found for position 1
21 unique residues found for position 2
21 unique residues found for position 3
21 unique residues found for position 4


In [62]:
print(len(mut_combo_dict),'unique mutation combinations found.')

1127 unique mutation combinations found.


## LkKDC library

In [63]:
file_dir = 'C:\\Users\\Joshua_Dietrich\\OneDrive - UW-Madison\\JD - Josh Dietrich\\Dietrich_E-Lab_Notebook\\Sequencing_results\\2023\\2023_1212_G4W\\Pfleger_G4W_raw_reads'
filename = 'Pfleger_G4W_1_LkKDC_lib.fastq'
reads = [r for r in SeqIO.parse(os.path.join(file_dir,filename),'fastq')]

In [64]:
Lkpos1 = 'ATTGTCAGAC' # upstream 10 bases of first mutation
Lkpos2 = 'GGGAACTTCT' # upstream 10 bases of second mutation
Lkpos3 = 'CGGCTATACC'
Lkpos4 = 'CTTACAAAAG'
Lk_positions = [Seq(Lkpos1),Seq(Lkpos2),Seq(Lkpos3),Seq(Lkpos4)]
min_qscore = 20 # min quality score for counting a mutation

In [65]:
found_codons = [dict() for p in Lk_positions]
residues = [dict() for p in Lk_positions]
read_dict = dict()
mut_combo_dict = dict()
for r in reads:
    # print(r.id)
    read_dict[r.id] = [None] * len(Lk_positions) # initialize a dict that maps a read to its mutated codons
    for idx,(p,d) in enumerate(zip(Lk_positions,found_codons)):
        # Loop through each mutation position
        if r.seq.find(p) != -1: # If the read actually has the current codon in it
            curr_read = r.seq
            curr_read_q_scores = r.letter_annotations["phred_quality"]
        elif r.seq.reverse_complement().find(p) != -1: # If the read is the reverse direction
            curr_read = r.seq.reverse_complement()
            curr_read_q_scores = list(reversed(r.letter_annotations["phred_quality"]))
        else:
            continue
        curr_codon_idx = curr_read.find(p)+len(p)
        curr_codon = curr_read[curr_codon_idx:curr_codon_idx+3]
        curr_q_scores = curr_read_q_scores[curr_codon_idx:curr_codon_idx+3]
        # print(r.id,idx,curr_codon_idx,curr_codon,curr_q_scores)
        # Check if the codon has a high enough q score, also checks if q scores correctly found
        if len(curr_q_scores) == 3 and min(curr_q_scores) >= min_qscore:
            # print(idx)
            # print(curr_codon,curr_q_scores)
            read_dict[r.id][idx] = str(curr_codon)
            if curr_codon in found_codons[idx]: # If the current codon has already been found
                found_codons[idx][str(curr_codon)] +=1
                residues[idx][str(Seq(curr_codon).translate())] +=1
            else:
                found_codons[idx][str(curr_codon)] = 1 # If the current codon has not yet been found
                residues[idx][str(Seq(curr_codon).translate())] = 1
    curr_mut_combo = ''.join([str(Seq(x).translate()) if x is not None else 'X' for x in read_dict[r.id]])
    if curr_mut_combo in mut_combo_dict:
        mut_combo_dict[curr_mut_combo] += 1
    else:
        mut_combo_dict[curr_mut_combo] = 1
mut_combo_dict.pop('XXXX',None) # Remove uninformative read data from count
    # print(idx,found_codons[idx],residues[idx])

184

In [66]:
for idx,d in enumerate(found_codons):
    print(len(d),'unique codons found for position',idx+1)

33 unique codons found for position 1
32 unique codons found for position 2
33 unique codons found for position 3
35 unique codons found for position 4


In [67]:
for idx,r in enumerate(residues):
    print(len(r),'unique residues found for position',idx+1)

21 unique residues found for position 1
21 unique residues found for position 2
21 unique residues found for position 3
21 unique residues found for position 4


In [68]:
print(len(mut_combo_dict),'unique mutation combinations found.')

1161 unique mutation combinations found.


In [70]:
mut_combo_dict

{'XXST': 2,
 'DXNV': 1,
 'LRAP': 1,
 'XXXN': 29,
 'SWXY': 1,
 'ATWH': 1,
 'XGCD': 1,
 'PXEL': 1,
 'LXTX': 1,
 'XSLX': 3,
 'RI*T': 1,
 'XXXT': 18,
 'XSLD': 1,
 'VPFL': 1,
 'KXXX': 2,
 'ICRX': 1,
 'GQLX': 1,
 'XXLY': 3,
 'RRXM': 1,
 'HCCX': 1,
 'XXX*': 6,
 'KRXT': 1,
 'XXXR': 16,
 'EXSX': 1,
 'XSSV': 1,
 'STIG': 1,
 'GXLQ': 1,
 'XXYD': 1,
 'RRXR': 1,
 'GXCX': 1,
 'LXVC': 1,
 'XXXS': 21,
 'XXPP': 1,
 'XXXK': 17,
 'XXXE': 8,
 'XRNX': 1,
 'RWVH': 1,
 'PRHN': 1,
 'KRLD': 1,
 'XAWA': 1,
 'XXWX': 2,
 'LXXC': 1,
 'PXXK': 1,
 'XXXQ': 8,
 'XALF': 1,
 'LXRG': 1,
 'XXMQ': 1,
 'LHSF': 1,
 'KXKF': 1,
 'APLX': 1,
 'VVLS': 1,
 'SSI*': 1,
 'XXWA': 1,
 'PELW': 1,
 'LXLL': 1,
 'GADX': 2,
 'GGTX': 2,
 'XXXW': 4,
 'SSLX': 1,
 'XGLA': 1,
 'WVSE': 1,
 'TAXN': 1,
 'XXTT': 2,
 'XXXM': 16,
 'XVXK': 1,
 'XGQT': 1,
 'LISG': 1,
 'XLGQ': 1,
 'XXXI': 28,
 'XKXN': 2,
 'XXXY': 5,
 'XXYP': 1,
 'FXCN': 1,
 'XAXX': 2,
 'XTFX': 1,
 'EVFK': 1,
 'VXFP': 1,
 'XTKP': 1,
 'H*LG': 1,
 'XXCR': 3,
 'RGXK': 2,
 'XASL': 1,
 'XTSC': 